# Introduction

LLM to be Fine Tuned on [SleepQA](https://github.com/IvaBojic/SleepQA/) dataset

[https://huggingface.co/EleutherAI/pythia-70m-deduped](https://huggingface.co/EleutherAI/pythia-70m-deduped)

The fine-tuned model can be found at -

[https://huggingface.co/ahmedshahriar/SleepQA-pythia-70m](https://huggingface.co/ahmedshahriar/SleepQA-pythia-70m)

# Libraries

In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q evaluate
# !pip install -q einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.7 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Sun Aug  6 07:28:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-c5db45b1-e645-f000-1c18-280bd29b5c46)


In [4]:
!lscpu |grep 'Model name'

Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz


In [5]:
#no.of cores each processor is having
!lscpu | grep 'Core(s) per socket:'

Core(s) per socket:              1


In [6]:
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'

Thread(s) per core:              2


In [7]:
#memory that we can use
!free -h --si | awk  '/Mem:/{print $2}'

12G


In [8]:
import re
import gc
import json
import math
import random
import torch

import collections
import numpy as np
import pandas as pd

from ast import literal_eval
from tqdm.auto import tqdm

from transformers import (pipeline, AutoTokenizer, AutoModelForCausalLM,
                          TrainingArguments, Trainer, DataCollatorForLanguageModeling,
                          )
from datasets import load_dataset
from evaluate import load, evaluator

from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import f1_score

SEED=42

def setting_seed(SEED):
  np.random.seed(SEED)
  random.seed(SEED)
  torch.manual_seed(SEED)

setting_seed(SEED)

In [9]:
df_sleep_train = pd.read_csv("https://raw.githubusercontent.com/IvaBojic/SleepQA/main/data/training/sleep-train.csv",
                 delimiter="\t",
                 header=None,
                 names=['question', 'answer'],
                #  converters={"answer": lambda x: x.strip('"[]"')}
                             )

In [10]:
df_sleep_train.answer

0           ["academic performance, behavior, and mood."]
1       ["can reveal whether someone's sleep problems ...
2       ["two pressure settings - inhalation positive ...
3       ["a firmer mattress and a pillow with a low lo...
4                               ["sleeping on the right"]
                              ...                        
3995    ["when the airflow from breathing causes flopp...
3996    ["shift workers who experience swd symptoms fo...
3997    ["mostly during the second half of the sleep p...
3998    ["they experienced an increase followed by a d...
3999    ["a decreased risk of stroke, heart attack, he...
Name: answer, Length: 4000, dtype: object

In [11]:
df_sleep_train.answer.apply(literal_eval).map(len).max()

1

In [12]:
# https://huggingface.co/docs/datasets/v1.1.2/loading_datasets.html#csv-files

base_url = "https://raw.githubusercontent.com/IvaBojic/SleepQA/main/data/training/"

dataset_sleep_raw = load_dataset("csv", data_files={"train": base_url + "sleep-train.csv",
                                          "validation": base_url + "sleep-dev.csv",
                                          "test": base_url + "sleep-test.csv"},
                       delimiter="\t",
                       header=None,
                       names=['question', 'answer'],
                       converters={"answer": lambda x: x.strip('"[]"')}
                      )

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [13]:
dataset_sleep_raw

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 500
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 500
    })
})

In [14]:
dataset_sleep_with_ctx = load_dataset("json", data_files={"train": base_url + "sleep-train.json",
                                          "validation": base_url + "sleep-dev.json",
                                          "test": base_url + "sleep-test.json"
                                          },
                      #  delimiter="\t",
                      #  header=None,
                      #  names=['question', 'answers'],
                      #  converters={"answers": lambda x: x.strip('"[]"')}
                      )

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [15]:
dataset_sleep_with_ctx

DatasetDict({
    train: Dataset({
        features: ['answers', 'negative_ctxs', 'question', 'positive_ctxs'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['answers', 'negative_ctxs', 'question', 'positive_ctxs'],
        num_rows: 500
    })
    test: Dataset({
        features: ['answers', 'negative_ctxs', 'question', 'positive_ctxs'],
        num_rows: 0
    })
})

In [16]:
dataset_sleep_raw['train']['question'][:5], dataset_sleep_raw['train']['answer'][:5]

(['what can lack of sleep in children impact?',
  'what is the purpose of the light sensor in the watch?',
  'how many pressure settings do bipap machines have?',
  'what do stomach sleepers tend to require?',
  'what can increase pressure on internal organs?'],
 ['academic performance, behavior, and mood.',
  "can reveal whether someone's sleep problems might be due to an overly bright bedroom or insufficient light during the day",
  'two pressure settings - inhalation positive airway pressure (ipap) and exhalation positive airway pressure (epap) - that allow for lower pressure levels during exhalation.',
  'a firmer mattress and a pillow with a low loft',
  'sleeping on the right'])

In [17]:
# Define a function to concatenate question and answer
def concat_question_answer(data):
    concatenated = "Given the question delimited by triple backticks \
                    ```{" + data['question'] + "}```, what is the answer? \
                    Answer: {" + data['answer'] + "}"
    return {"text": concatenated}

# Apply the function to create the new column
dataset_sleep = dataset_sleep_raw.map(concat_question_answer)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [29]:
# Loading model and tokenizer

# pythia-70m

model_name = "EleutherAI/pythia-70m-deduped"
model_suffix = "pythia-70m"

from transformers import GPTNeoXForCausalLM, AutoTokenizer

model = GPTNeoXForCausalLM.from_pretrained(
  model_name,
  revision="step3000",
  cache_dir="./pythia-70m-deduped/step3000",
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(
  model_name,
  revision="step3000",
  cache_dir="./pythia-70m-deduped/step3000",
)

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

In [30]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

Using pad_token, but it is not set yet.


In [31]:
# # prepare the data for training
def prepare_train_data(data):
    # prompt + completion
    text_input = data['text']
    # tokenize the input (prompt + completion) text
    tokenized_input = tokenizer(text_input,
                                return_tensors='pt',
                                padding='max_length',
                                truncation=True,
                                max_length=512
                                )
    # generative models: labels are the same as the input
    tokenized_input['labels'] = tokenized_input['input_ids']
    return tokenized_input

dataset_sleep_tokenized = dataset_sleep.map(prepare_train_data,
                                     batched=True,
                                     remove_columns=dataset_sleep["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [32]:
dataset_sleep_tokenized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
})

In [33]:
len(dataset_sleep_tokenized["train"]['input_ids']), len(dataset_sleep['train'])

(4000, 4000)

In [34]:
training_arguments = TrainingArguments(
    'SleepQA-'+model_suffix,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    optim="adafactor",
    gradient_accumulation_steps=4,
    gradient_checkpointing=True
)

# Training

In [35]:
%%time

# The operator 'aten::cumsum.out' is not currently supported on the MPS
# https://stackoverflow.com/a/72416727/11105356

trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset=dataset_sleep_tokenized["train"], # .select(range(100))
    eval_dataset=dataset_sleep_tokenized["validation"], # .select(range(50))
    # data_collator=data_collator,
)

trainer.train()
trainer.save_model()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
500,0.374500
1000,0.142800


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...

CPU times: user 5min 35s, sys: 3.92 s, total: 5min 39s
Wall time: 6min 37s


# Evaluation Training

In [36]:
%%time

trainer.evaluate()

CPU times: user 4.96 s, sys: 23 ms, total: 4.98 s
Wall time: 5.01 s


{'eval_loss': 0.15372134745121002,
 'eval_runtime': 5.0096,
 'eval_samples_per_second': 99.808,
 'eval_steps_per_second': 12.576,
 'epoch': 2.0}

In [37]:
# perplexity
# https://huggingface.co/docs/transformers/v4.31.0/en/tasks/language_modeling#train

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.4f}")

Perplexity: 1.1662


In [38]:
dataset_sleep_tokenized['validation'].select(range(5))

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5
})

In [39]:
trainer.args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=4,
gradient_checkpointing=True,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_privat

In [40]:
import gc

gc.collect()
torch.cuda.empty_cache()

# Evaluation Text Generation

## Original Model

In [41]:
model_old = GPTNeoXForCausalLM.from_pretrained(
  model_name,
  revision="step3000",
  # cache_dir="./pythia-70m-deduped/step3000",
  low_cpu_mem_usage=True
).to("cpu")


prompt = 'Given the question delimited by triple backticks \
          ```{what do stomach sleepers tend to require}```, what is the answer? \
          Answer:'
generator = pipeline('text-generation',
                     model=model_old,
                     tokenizer=tokenizer,
                     do_sample=False)
result = generator(prompt)
print(result)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Input length of input_ids is 32, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


[{'generated_text': 'Given the question delimited by triple backticks           ```{what do stomach sleepers tend to require}```, what is the answer?           Answer:  '}]


## Fine Tuned Model

In [45]:
model_sleep = AutoModelForCausalLM.from_pretrained("SleepQA-pythia-70m",
                                          low_cpu_mem_usage=True).to("cpu")

prompt = 'Given the question delimited by triple backticks \
          ```{what do stomach sleepers tend to require}```, what is the answer? \
          Answer:'

generator = pipeline('text-generation',
                      model=model_sleep,
                      tokenizer=tokenizer,
                      do_sample=False)
result = generator(prompt, max_length=128)
display(result)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Given the question delimited by triple backticks           ```{what do stomach sleepers tend to require}```, what is the answer?           Answer: {sleeping on the back}stones}```, what is the answer?}```, what is the answer?           Answer: {sleeping on the back}stones}stones}stones}stonesmithsmithsmithsmithsmithsmithsmithsmithsmithsmithsmithsmiths'}]

### Push To Hub

In [46]:
from huggingface_hub import login, Repository, get_full_repo_name

login()

In [48]:
model_name = "SleepQA-pythia-70m"
repo_name = get_full_repo_name(model_name)
display(repo_name)

tokenizer.save_pretrained(model_name)
model_sleep.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

'ahmedshahriar/SleepQA-pythia-70m'

pytorch_model.bin:   0%|          | 0.00/282M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ahmedshahriar/SleepQA-pythia-70m/commit/d901465c605035daa5f430b6e054f87ed267eea4', commit_message='Upload tokenizer', commit_description='', oid='d901465c605035daa5f430b6e054f87ed267eea4', pr_url=None, pr_revision=None, pr_num=None)

### Sample QA

In [50]:
prompt = 'what do stomach sleepers tend to require?'

generator = pipeline('text-generation',
                      model=model_sleep,
                      tokenizer=tokenizer,
                      do_sample=False)
result = generator(prompt, max_length=32)
display(result)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'what do stomach sleepers tend to require?\n<brianne> brianne: what is the answer?\n<brianne> brian'}]

##### Train Set

In [55]:
# https://huggingface.co/learn/nlp-course/chapter5/3#slicing-and-dicing-our-data

sample_train_data = dataset_sleep['train'].shuffle(seed=SEED).select(range(10))

for example in sample_train_data:
  prompt = 'Given the question delimited by triple backticks \
          ```{'+example['question']+'}```, what is the answer? \
          Answer:'
  true_ans = example['answer']

  generator = pipeline('text-generation',
                        model=model_sleep,
                        tokenizer=tokenizer,
                        do_sample=False)
  result = generator(prompt, max_length=64)
  display(result, '\n\n', 'True Answer:', true_ans)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': "Given the question delimited by triple backticks           ```{what events may affect our dreams?}```, what is the answer?           Answer: {the brain's ability to control the brain's ability to control the brain's ability to control the brain's ability to control the brain's ability to control the brain"}]

'\n\n'

'True Answer:'

'major life changes, such as pregnancy or trauma'

...

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Given the question delimited by triple backticks           ```{how does fragmented sleep affect memory?}```, what is the answer?           Answer: {by reducing the amount of time you spend in bed, and reducing the amount of time you spend in bed}'}]

'\n\n'

'True Answer:'

'negatively affect memory'

##### Validation Set

In [56]:
# https://huggingface.co/learn/nlp-course/chapter5/3#slicing-and-dicing-our-data

sample_train_data = dataset_sleep['validation'].shuffle(seed=SEED).select(range(10))

for example in sample_train_data:
  prompt = 'Given the question delimited by triple backticks \
          ```{'+example['question']+'}```, what is the answer? \
          Answer:'
  true_ans = example['answer']

  generator = pipeline('text-generation',
                        model=model_sleep,
                        tokenizer=tokenizer,
                        do_sample=False)
  result = generator(prompt, max_length=64)
  display(result, '\n\n', 'True Answer:', true_ans)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': "Given the question delimited by triple backticks           ```{how does a stuffy nose during sleep lead to snoring?}```, what is the answer?           Answer: {a person's nose, nose, and nose}"}]

'\n\n'

'True Answer:'

'by reducing the flow of air through the airway and causing the airway to collapse'

...

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': "Given the question delimited by triple backticks           ```{how does sleep talking affect a person's sleep?}```, what is the answer?           Answer: {by talking to a person's sleep partner}stones"}]

'\n\n'

'True Answer:'

"it doesn't usually have a major effect on the person's sleep"

#### Test Set

In [57]:
# https://huggingface.co/learn/nlp-course/chapter5/3#slicing-and-dicing-our-data

sample_train_data = dataset_sleep['test'].shuffle(seed=SEED).select(range(10))

for example in sample_train_data:
  prompt = 'Given the question delimited by triple backticks \
          ```{'+example['question']+'}```, what is the answer? \
          Answer:'
  true_ans = example['answer']

  generator = pipeline('text-generation',
                        model=model_sleep,
                        tokenizer=tokenizer,
                        do_sample=False)
  result = generator(prompt, max_length=64)
  display(result, '\n\n', 'True Answer:', true_ans)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Given the question delimited by triple backticks           ```{what is talalay latex typically reserved for in mattresses?}```, what is the answer?           Answer: {a latex that is placed in a mattress}'}]

'\n\n'

'True Answer:'

'comfort layers in mattresses'

...

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'Given the question delimited by triple backticks           ```{what does sleep affect?}```, what is the answer?           Answer: {a sleep-related sleep disorder} Vaishammeryammeryam'}]

'\n\n'

'True Answer:'

'various aspects of your overall health, from your mood to your immune system'

# Evaluation QA

In [58]:
'max answer length', max((len(l) for l in dataset_sleep_with_ctx['train']['answers']))

# for example in dataset_sleep_with_ctx['validation']:
#     context = example['positive_ctxs']
#     print(context[0]['text'])

('max answer length', 1)

## Train

In [59]:
%%time

# helper function
# https://github.com/fastforwardlabs/ff14_blog/blob/master/_notebooks/2020-06-09-Evaluating_BERT_on_SQuAD.ipynb

# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

# Load pre-trained question answering model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m-deduped")
qa_pipeline = pipeline("question-answering",
                       model="SleepQA-pythia-70m",  # distilbert-base-cased-distilled-squad SleepQA-palmyra-small
                       tokenizer=tokenizer, device='cuda')

predicted_answers = []
true_answers = []

# Evaluate the model on the test data
for example in dataset_sleep_with_ctx['train']:
    # question = 'Given the question delimited by triple backticks \
    #       ```{'+ example['question'] +'}```, what is the answer? \
    #       Answer:'

    question = example['question']
    context = example['positive_ctxs'][0]['text'] + example['negative_ctxs'][0]['text']
    true_answer = example['answers'][0]


    # Tokenize the input
    # inputs = tokenizer.encode_plus(question, context, return_tensors='pt')

    # outputs = model_sleep(**inputs)
    # answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
    # answer_end = torch.argmax(outputs[1]) + 1

    # answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))


    # Make predictions
    # print(question, context)
    answer = qa_pipeline(question, context)

    # Append predicted and true answers
    predicted_answers.append(answer["answer"])
    true_answers.append(true_answer)

    # Print the predicted answer

    # display(f"Question: {question}")
    # display(f"Context: {context}")
    # display(f"Predicted Answer: {answer['answer']}\n")
    # display(f"True Answer: {true_answer}\n")


# Calculate F1 score
f1 = f1_score([normalize_text(ans) for ans in true_answers], [normalize_text(ans) for ans in predicted_answers], average="macro")

# Calculate exact match (EM) score
exact_match = sum(1 for true, pred in zip(true_answers, predicted_answers) if normalize_text(true) == normalize_text(pred)) / len(true_answers)

display(f"F1 Score: {f1:.4f}")
display(f"Exact Match (EM) Score: {exact_match:.4f}")

Some weights of GPTNeoXForQuestionAnswering were not initialized from the model checkpoint at SleepQA-pythia-70m and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'F1 Score: 0.0007'

'Exact Match (EM) Score: 0.0015'

CPU times: user 1min 3s, sys: 532 ms, total: 1min 4s
Wall time: 1min 5s


## Validation

In [60]:
%%time

# helper function
# https://github.com/fastforwardlabs/ff14_blog/blob/master/_notebooks/2020-06-09-Evaluating_BERT_on_SQuAD.ipynb

# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

# Load pre-trained question answering model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m-deduped")
qa_pipeline = pipeline("question-answering",
                       model="SleepQA-pythia-70m",  # distilbert-base-cased-distilled-squad SleepQA-palmyra-small
                       tokenizer=tokenizer, device='cuda')

predicted_answers = []
true_answers = []

# Evaluate the model on the test data
for example in dataset_sleep_with_ctx['validation']:
    # question = 'Given the question delimited by triple backticks \
    #       ```{'+ example['question'] +'}```, what is the answer? \
    #       Answer:'

    question = example['question']
    context = example['positive_ctxs'][0]['text'] + example['negative_ctxs'][0]['text']
    true_answer = example['answers'][0]


    # Tokenize the input
    # inputs = tokenizer.encode_plus(question, context, return_tensors='pt')

    # outputs = model_sleep(**inputs)
    # answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
    # answer_end = torch.argmax(outputs[1]) + 1

    # answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))


    # Make predictions
    # print(question, context)
    answer = qa_pipeline(question, context)

    # Append predicted and true answers
    predicted_answers.append(answer["answer"])
    true_answers.append(true_answer)

    # Print the predicted answer

    # display(f"Question: {question}")
    # display(f"Context: {context}")
    # display(f"Predicted Answer: {answer['answer']}\n")
    # display(f"True Answer: {true_answer}\n")


# Calculate F1 score
f1 = f1_score([normalize_text(ans) for ans in true_answers], [normalize_text(ans) for ans in predicted_answers], average="macro")

# Calculate exact match (EM) score
exact_match = sum(1 for true, pred in zip(true_answers, predicted_answers) if normalize_text(true) == normalize_text(pred)) / len(true_answers)

display(f"F1 Score: {f1:.4f}")
display(f"Exact Match (EM) Score: {exact_match:.4f}")

Some weights of GPTNeoXForQuestionAnswering were not initialized from the model checkpoint at SleepQA-pythia-70m and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'F1 Score: 0.0000'

'Exact Match (EM) Score: 0.0000'

CPU times: user 9.27 s, sys: 244 ms, total: 9.51 s
Wall time: 9.84 s
